In [1]:
from scqbf.scqbf_evaluator import *
from scqbf.scqbf_solution import *
from scqbf.scqbf_ts import *
from scqbf.scqbf_instance import *

import pandas as pd
import numpy as np
import pickle

In [2]:
instance_paths = [(f"instances/gen{i}/instance{j}.txt", i, j) for i in range(1, 4) for j in range(1, 6)]

In [3]:
tenure_1 = lambda x: math.ceil(math.sqrt(x))
tenure_2 = lambda x: min(math.ceil(x/4), 20) # Inspired by MATLAB's default (https://in.mathworks.com/help/matlab/math/solve-qubo-tabu-search.html)
tenure_3 = lambda x: 10 # Fixed small tenure (Extra)

In [4]:
def run_experiment(config: dict, tenure_func) -> pd.DataFrame:
    results = []
    exp_num = 1

    for instance_path, gen, inst in instance_paths:
        instance = read_max_sc_qbf_instance(instance_path)
        print(f"{exp_num}: {inst}th instance of generation strategy {gen}. Path: {instance_path}")
        exp_num += 1
        
        time_limit = 60*30
        tenure = tenure_func(instance.n)
        tabu_search = ScQbfTabuSearch(instance, config=config, max_iterations=None, time_limit_secs=time_limit, tenure=tenure, patience=1000)
        best_solution = tabu_search.solve()

        if tabu_search.solve_time >= time_limit:
            stop_reason = "time_limit"
        else:
            stop_reason = "patience_exceeded" 
        
        evaluator = ScQbfEvaluator(instance)
        
        results.append({
            'gen': gen,
            'inst': inst,
            'n': instance.n,
            'stop_reason': stop_reason,
            'best_objective': evaluator.evaluate_objfun(best_solution),
            'coverage': evaluator.evaluate_coverage(best_solution),
            'time_taken': tabu_search.solve_time
        })
        
        last_result = results[-1]
        print(f"\tBest objective value: {last_result['best_objective']:.4f}")
        print(f"Selected elements: {best_solution.elements}")
        print(f"\tCoverage: {last_result['coverage']:.2%}")
        print(f"\tTime taken (secs): {last_result['time_taken']:.4f} s")
        print(f"\tStop reason: {last_result['stop_reason']}")
        print()
    
    df = pd.DataFrame(results)
    return df

## Configuração 1 - PADRÃO:
GRASP com parâmetro α1 , first-improving e heurística construtiva padrão.

In [5]:
config_1 = {
    "local_search_method": "best_improve",
    "diversification_method": "infrequent_elements",
}
results_df = run_experiment(config_1, tenure_1)

with open("pickles/results_config_1.pkl", "wb") as f:
    pickle.dump(results_df, f)

display(results_df)

1: 1th instance of generation strategy 1. Path: instances/gen1/instance1.txt


KeyboardInterrupt: 